In [1]:
import pandas as pd
import os
import re
import numpy as np
import ujson as json
import dill
import pickle
from bottleneck import push

In [2]:
data_path = 'physionet'
with open(os.path.join(data_path, 'FP_2012_origional.pkl'), 'rb') as f:
    origional = pickle.load(f)

In [3]:
origional.shape

(4000, 48, 35)

Functions used to normalize and renomrmalize the data

In [4]:
def renormalize(data, means, stds):
    renorms=[]
    for patient in data:
        r= patient*stds+ means
        renorms.append(r)
    renorm_full = np.stack(renorms, axis=0)
    return(renorm_full)
def normalize(data, means, stds):
    renorms=[]
    for patient in data:
        r= (patient-means) /stds
        renorms.append(r)
    renorm_full = np.stack(renorms, axis=0)
    return(renorm_full)

In [5]:
def create_individualized_missingness_mask(mask):
  np.set_printoptions(suppress=False, precision= 9)
  samples_len =mask.shape[0]
  time_steps = mask.shape[1]
  features = mask.shape[2]
  
  personalized_mask_full = np.empty(shape=[samples_len,time_steps,features])
  personalized_mask_patient = []
  personalized_mask_sample = np.ones(shape=[time_steps,features])
  for patient_mask in mask:
        num_measurments_per_feature = patient_mask.sum(axis=0)
        # for each patient mask
        tf=((num_measurments_per_feature)/time_steps)
        personalized_mask_patient.append(np.where(patient_mask == 0, tf, patient_mask))
    # stack all feature-specific patient masks tnto a 3d tensor
  personalized_mask_full = np.stack(personalized_mask_patient, axis=0)
  return(personalized_mask_full)

Create Binary Mask

In [6]:
mask=~np.isnan(origional)*1

Create Individualized Missingness Mask (IMM)

In [7]:
IMM= create_individualized_missingness_mask(mask)

Last Observation Carried Forward (LOCF), 
all missing values with no prior measurments, are replaced with zero

In [8]:
LOCV =push(origional, axis=1)
LOCV= np.where(np.isnan(LOCV), 0, LOCV)

Zero Imputation

In [9]:
zero= np.where(np.isnan(origional), 0, origional)